# stLearn - Core plotting functions

Here we want to introduce several visualization functions in stLearn.

Source: https://www.10xgenomics.com/datasets/human-breast-cancer-block-a-section-1-1-standard-1-1-0


### Loading processed data

In [ ]:
import stlearn as st
import pathlib as pathlib
import pandas as pd

st.settings.set_figure_params(dpi=120)

# Ignore all warnings
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Read raw data
# Setup download directory and get data
st.settings.datasetdir =  pathlib.Path.cwd().parent / "data"
sample_id = "V1_Breast_Cancer_Block_A_Section_1"
adata = st.datasets.visium_sge(sample_id=sample_id)
adata = st.convert_scanpy(adata)

In [ ]:
# Adding the label transfer results
project_root = pathlib.Path.cwd().parent
annotation_path = project_root / "annotations"
cell_types = annotation_path / f"{sample_id}_cell_type_proportions.csv"
spot_mixtures = pd.read_csv(cell_types, index_col=0)
aligned_spot_mixtures = spot_mixtures.reindex(adata.obs_names, fill_value=0)
labels = aligned_spot_mixtures.idxmax(axis=1)
labels.name = "cell_type"
adata.obs['cell_type'] = labels # Adding the dominant cell type labels per spot
adata.obs['cell_type'] = adata.obs['cell_type'].astype('category')
adata.uns['cell_type'] = aligned_spot_mixtures

In [ ]:
adata_processed = adata.copy()
st.pp.filter_genes(adata_processed, min_cells=3)
st.pp.normalize_total(adata_processed)
st.pp.log1p(adata_processed)

In [ ]:
st.em.run_pca(adata_processed, n_comps=50, random_state=0)
st.pp.neighbors(adata_processed, n_neighbors=25, use_rep='X_pca', random_state=0)
st.tl.clustering.louvain(adata_processed, resolution=1.15, random_state=0)

### Gene plot

Here is the standard plot for gene expression, we provide 2 options for single genes and multiple genes:

In [ ]:
st.pl.gene_plot(adata, gene_symbols="BRCA1")

For multiple genes, you can combine multiple genes by `'CumSum'`cummulative sum or `'NaiveMean'`naive mean:

In [ ]:
st.pl.gene_plot(adata, gene_symbols=["BRCA1","BRCA2"], method="CumSum")

In [ ]:
st.pl.gene_plot(adata, gene_symbols=["BRCA1","BRCA2"], method="NaiveMean")

You also can plot genes with contour plot to see clearer about the distribution of genes:

In [ ]:
st.pl.gene_plot(adata, gene_symbols="GAPDH", contour=True,cell_alpha=0.5)

You can change the `step_size` to cut the range of display in contour

In [ ]:
st.pl.gene_plot(adata, gene_symbols="GAPDH", contour=True,cell_alpha=0.5, step_size=200)

### Cluster plot

We provide different options for display clustering results. Several `show_*` options that user can control to display different parts of the figure:

In [ ]:
st.pl.cluster_plot(adata_processed, use_label="louvain")

In [ ]:
st.pl.cluster_plot(adata_processed, use_label="louvain", show_cluster_labels=True, show_color_bar=False)

### Subcluster plot

We also provide option to plot spatial subclusters based on the spatial location within a cluster.

You have two options here, display subclusters for multiple clusters using `show_subcluster` in `st.pl.cluster_plot` or use `st.pl.subcluster_plot` to display subclusters within a cluster but with different color.

In [ ]:
# Generate subclusters with a distance of 50
st.spatial.clustering.localization(adata_processed, eps=50)

In [ ]:
st.pl.cluster_plot(adata_processed, use_label="louvain", show_subcluster=True, show_color_bar=False, list_clusters=["6","7"])

In [ ]:
st.pl.subcluster_plot(adata_processed, use_label="louvain", cluster="6")

### Spatial trajectory plot

We provided `st.pl.trajectory.pseudotime_plot` to visualize PAGA graph that maps into spatial transcriptomics array.

In [ ]:
adata_processed.raw = adata
adata_processed.uns["iroot"] = st.spatial.trajectory.set_root(adata_processed, use_label = "louvain", cluster = "6", use_raw=True)
st.spatial.trajectory.pseudotime(adata_processed, eps = 50, n_neighbors = 30, use_rep="X_pca", use_label = "louvain")
adata_processed.uns['available_paths']

In [ ]:
st.pl.trajectory.pseudotime_plot(adata_processed, use_label="louvain", pseudotime_key="dpt_pseudotime", list_clusters=["6","7"], show_node=True)

In [ ]:
st.spatial.trajectory.pseudotimespace_global(adata_processed, use_label = "louvain", list_clusters = ["6","7"])

You can plot spatial trajectory analysis results with the node in each subcluster by `show_trajectories` and `show_node` parameters.

In [ ]:
st.pl.cluster_plot(adata_processed,use_label="louvain",show_trajectories=True, show_color_bar=True, list_clusters=["6","7"], show_node=True)

### Ligand-receptor interaction plots

For the stLearn ligand-receptor cell-cell interaction analysis, you can display basic results for LRs using `st.pl.lr_result_plot`. For many more visualisations, please see the stLearn Cell-cell interaction analysis tutorial.

In [ ]:
lrs = st.tl.cci.load_lrs(['connectomeDB2020_lit'], species='human')
lrs

In [ ]:
# Running the analysis #
st.tl.cci.run(adata_processed, lrs,
                  min_spots = 20, #Filter out any LR pairs with no scores for less than min_spots
                  distance=None, # None defaults to spot+immediate neighbours; distance=0 for within-spot mode
                  n_pairs=100, # Number of random pairs to generate; low as example, recommend ~10,000
                  n_cpus=4, # Number of CPUs for parallel. If None, detects & use all available.
                  )

In [ ]:
st.pl.lr_summary(adata_processed, highlight_lrs=['GPC3_IGF1R'])

In [ ]:
st.pl.lr_result_plot(adata_processed, "GPC3_IGF1R", "-log10(p_adjs)")

In [ ]:
st.pl.lr_result_plot(adata_processed, "GPC3_IGF1R", "lr_sig_scores")

### Cell-cell interaction plots
For the stLearn cell-cell interaction analysis, you can display the celltype-celltype interactions between cell types using `st.pl.lr_chord_plot`. 

In [ ]:
st.pl.cluster_plot(adata_processed, use_label='cell_type')
st.pl.lr_chord_plot(adata_processed, 'cell_type', 'GPC3_IGF1R', figsize=(4,4))